In [101]:
from mxnet import autograd,nd
from mxnet.gluon import nn

## Convolution

In [102]:
def corr2d(X,K):
    Y=nd.zeros(shape=(X.shape[0]-K.shape[0]+1,X.shape[1]-K.shape[1]+1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i,j]=(X[i:i+K.shape[0],j:j+K.shape[1]]*K).sum()
    return Y

## detect(apply)

In [103]:
X = nd.ones((6, 8))
X[:, 2:6] = 0
K = nd.array([[1, -1]])
Y = corr2d(X, K)
X,Y

(
 [[1. 1. 0. 0. 0. 0. 1. 1.]
  [1. 1. 0. 0. 0. 0. 1. 1.]
  [1. 1. 0. 0. 0. 0. 1. 1.]
  [1. 1. 0. 0. 0. 0. 1. 1.]
  [1. 1. 0. 0. 0. 0. 1. 1.]
  [1. 1. 0. 0. 0. 0. 1. 1.]]
 <NDArray 6x8 @cpu(0)>, 
 [[ 0.  1.  0.  0.  0. -1.  0.]
  [ 0.  1.  0.  0.  0. -1.  0.]
  [ 0.  1.  0.  0.  0. -1.  0.]
  [ 0.  1.  0.  0.  0. -1.  0.]
  [ 0.  1.  0.  0.  0. -1.  0.]
  [ 0.  1.  0.  0.  0. -1.  0.]]
 <NDArray 6x7 @cpu(0)>)

## learn the K

In [104]:
conv2d=nn.Conv2D(1,kernel_size=(1,2))
conv2d.initialize()
X=X.reshape(1,1,6,-1)
Y=Y.reshape(1,1,6,-1)
for i in range(10):
    with autograd.record():
        Y_pred=conv2d(X)
        l=(Y_pred-Y)**2
    l.backward()
    conv2d.weight.data()[:]=conv2d.weight.data()-3e-2*conv2d.weight.grad()
print("epoch %d,loss %.3f"%(i+1,l.sum().asscalar()))
print(conv2d.weight.data())

epoch 10,loss 0.004

[[[[ 0.9894602 -0.9874791]]]]
<NDArray 1x1x1x2 @cpu(0)>


## padding

In [105]:
def  comp_conv2d(conv2d,X):
    conv2d.initialize()
    X=X.reshape((1,1)+X.shape)
    Y=conv2d(X)
    return Y.shape[2:]
X=nd.random.uniform(shape=(6,6))
conv2d=nn.Conv2D(1,kernel_size=3,padding=1)
comp_conv2d(conv2d,X)

(6, 6)

## stride

In [106]:
conv2d=nn.Conv2D(1,kernel_size=3,padding=1,strides=2)
comp_conv2d(conv2d,X)

(3, 3)

## channel

In [107]:
import gluonbook as gb
def corr2d_multi_in(X,K):
    return nd.add_n(*[corr2d(x,k) for x,k in zip(X,K)])

In [108]:
X=nd.array([[[0,1,2],[3,4,5],[6,7,8]],
            [[1,2,3],[4,5,6],[7,8,9]]])
K=nd.array([[[0,1],[2,3]],[[1,2],[3,4]]])
X.shape,K.shape,corr2d_multi_in(X,K).shape

((2, 3, 3), (2, 2, 2), (2, 2))

In [109]:
def corr2d_multi_in_out(X,K):
    return nd.stack(*[corr2d_multi_in(X,k) for k in K])

In [110]:
K=nd.array([[[0,1],[2,3]],[[1,2],[3,4]]])
K=nd.stack(K,K+1,K+2)
X.shape,K.shape,corr2d_multi_in_out(X,K).shape

((2, 3, 3), (3, 2, 2, 2), (3, 2, 2))

##  pooling

In [111]:
def pool2d(X,pool_size,mode='max'):
    Y=nd.ones(shape=(X.shape[0]-pool_size[0]+1,X.shape[1]+1-pool_size[1]))
    for  i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            if mode == 'max':
                Y[i,j]=X[i:i+pool_size[0],j:j+pool_size[1]].max()
            elif mode =='avg':
                Y[i,j]=X[i:i+pool_size[0],j:j+pool_size[1]].mean()
    return Y

In [112]:
X=nd.array([[0,1,2],[3,4,5],[6,7,8]])
pool2d(X,(2,2))


[[4. 5.]
 [7. 8.]]
<NDArray 2x2 @cpu(0)>

In [113]:
X=nd.arange(16).reshape((1,1,4,-1))
pool2d=nn.MaxPool2D(pool_size=3,padding=1)
pool2d(X)


[[[[ 5.  7.]
   [13. 15.]]]]
<NDArray 1x1x2x2 @cpu(0)>